## Cleaning and Structuring

In [1]:
import pandas as pd

# Load raw Excel file
df_raw = pd.read_excel('Seasonality_Model.xlsx')

# Step 1: Rename columns
df_raw.columns = ['Restaurant Name', 'Go Live Date', 'Transaction Date', 'GMV', 'Revenue']

# Step 2: Convert date columns
df_raw['Go Live Date'] = pd.to_datetime(df_raw['Go Live Date'], errors='coerce')
df_raw['Transaction Date'] = pd.to_datetime(df_raw['Transaction Date'], errors='coerce')

# Step 3: Clean GMV and Revenue columns (remove commas, convert to float)
df_raw['GMV'] = df_raw['GMV'].replace(',', '', regex=True).astype(float)
df_raw['Revenue'] = df_raw['Revenue'].replace(',', '', regex=True).astype(float)

# Step 4: Drop rows with missing GMV or Revenue
df_cleaned = df_raw.dropna(subset=['GMV', 'Revenue']).reset_index(drop=True)

# Step 5: Save cleaned dataset to Excel
df_cleaned.to_excel('Cleaned_Seasonality_Data.xlsx', index=False)

print("✅ Cleaned dataset saved to 'Cleaned_Seasonality_Data.xlsx'")


✅ Cleaned dataset saved to 'Cleaned_Seasonality_Data.xlsx'


In [2]:
df[df['GMV'].isnull() | df['Revenue'].isnull()]
df = df.dropna(subset=['GMV', 'Revenue']).reset_index(drop=True)


## Calculate Seasonality Indices

In [7]:
import pandas as pd

# Step 1: Load cleaned data
df = pd.read_excel('Cleaned_Seasonality_Data.xlsx')

# Step 2: Create Month column
df['Month'] = df['Transaction Date'].dt.month
df['Year'] = df['Transaction Date'].dt.year
df['Month-Year'] = df['Transaction Date'].dt.to_period('M')

# Step 3: Calculate monthly revenue per restaurant
monthly_revenue = df.groupby(['Restaurant Name', 'Month'])['Revenue'].mean().reset_index()
monthly_revenue.rename(columns={'Revenue': 'Avg Monthly Revenue'}, inplace=True)

# Step 4: Calculate overall average revenue per restaurant
overall_avg = df.groupby('Restaurant Name')['Revenue'].mean().reset_index()
overall_avg.rename(columns={'Revenue': 'Overall Avg Revenue'}, inplace=True)

# Step 5: Merge and calculate seasonality index
seasonality_df = pd.merge(monthly_revenue, overall_avg, on='Restaurant Name')
seasonality_df['Seasonality Index'] = seasonality_df['Avg Monthly Revenue'] / seasonality_df['Overall Avg Revenue']

# Step 6: Round for readability
seasonality_df = seasonality_df[['Restaurant Name', 'Month', 'Avg Monthly Revenue', 'Seasonality Index']]
seasonality_df = seasonality_df.sort_values(['Restaurant Name', 'Month'])
seasonality_df['Avg Monthly Revenue'] = seasonality_df['Avg Monthly Revenue'].round(2)
seasonality_df['Seasonality Index'] = seasonality_df['Seasonality Index'].round(3)

# Step 7: Save to Excel
seasonality_df.to_excel('Monthly_Seasonality_Indices.xlsx', index=False)

print("✅ Monthly seasonality indices saved to 'Monthly_Seasonality_Indices.xlsx'")


✅ Monthly seasonality indices saved to 'Monthly_Seasonality_Indices.xlsx'


## Forecast next 6 months of revenue for a selected restaurant

In [10]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Step 1: Load cleaned data
df = pd.read_excel('Cleaned_Seasonality_Data.xlsx')

# Step 2: Extract Month-Year
df['Month-Year'] = df['Transaction Date'].dt.to_period('M')
df['Month-Year'] = df['Month-Year'].dt.to_timestamp()

# Step 3: Get list of restaurants
restaurants = df['Restaurant Name'].unique()

# Step 4: Prepare forecast result list
forecast_records = []

# Step 5: Loop through each restaurant
for restaurant in restaurants:
    venue_df = df[df['Restaurant Name'] == restaurant]

    # Monthly total revenue
    monthly = venue_df.groupby('Month-Year')['Revenue'].sum().sort_index()

    if len(monthly) < 3:
        continue  # Not enough data

    # Dynamic seasonality
    if len(monthly) >= 24:
        seasonal_periods = 12
    elif len(monthly) >= 12:
        seasonal_periods = 6
    else:
        seasonal_periods = None

    try:
        if seasonal_periods:
            model = ExponentialSmoothing(
                monthly,
                trend='add',
                seasonal='add',
                seasonal_periods=seasonal_periods
            ).fit()
        else:
            model = ExponentialSmoothing(
                monthly,
                trend='add',
                seasonal=None
            ).fit()

        # Forecast next 6 months
        forecast = model.forecast(6)

        # Store forecast
        for date, value in forecast.items():
            forecast_records.append({
                'Restaurant Name': restaurant,
                'Forecast Month': date.strftime('%Y-%m'),
                'Forecasted Revenue': round(value, 2)
            })

    except:
        continue  # Skip if model fails

# Step 6: Save to Excel
forecast_df = pd.DataFrame(forecast_records)
forecast_df.to_excel('6_Month_Revenue_Forecast.xlsx', index=False)

print("✅ Forecasted revenue saved to '6_Month_Revenue_Forecast.xlsx'")


E:\DS_setup\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
E:\DS_setup\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
E:\DS_setup\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
E:\DS_setup\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
E:\DS_setup\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
E:\DS_setup\Lib\site-packages\stats

✅ Forecasted revenue saved to '6_Month_Revenue_Forecast.xlsx'


E:\DS_setup\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


In [5]:
import pandas as pd
import numpy as np

# Load existing files
scores = pd.read_excel('Seasonality_Scores.xlsx')
indices = pd.read_excel('Monthly_Seasonality_Indices.xlsx')

# Peak months
peak_months = (
    indices.sort_values(['Restaurant Name', 'Seasonality Index'], ascending=False)
    .groupby('Restaurant Name')
    .head(2)
    .groupby('Restaurant Name')['Month']
    .apply(lambda x: ', '.join(pd.to_datetime(x, format='%m').dt.strftime('%b')))
    .reset_index()
    .rename(columns={'Month': 'Peak Months'})
)

# Merge base summary
summary = pd.merge(scores, peak_months, on='Restaurant Name', how='left')

def safe_suggest_action(row):
    score = row['Seasonality Score']
    months = row['Peak Months']

    if pd.isna(score) or pd.isna(months):
        return "Insufficient data"

    score = int(score)
    if score >= 4:
        return f"Heavy variability — staff up in {months}, promote during lows."
    elif score == 3:
        return f"Moderate swings — prepare lightly for {months} peaks."
    else:
        return f"Stable venue — focus on upselling and loyalty programs."

summary['Suggested Action'] = summary.apply(safe_suggest_action, axis=1)

# Executive Overview
overview = {
    "Topic": [
        "Total Venues Analyzed",
        "Data Range",
        "Avg Seasonality Score",
        "High Seasonality Venues",
        "Stable Venues",
        "Forecast Scope"
    ],
    "Insight": [
        len(summary),
        "From earliest Go Live to latest transaction (daily granularity)",
        round(summary['Seasonality Score'].mean(), 2),
        (summary['Seasonality Score'] >= 4).sum(),
        (summary['Seasonality Score'] == 1).sum(),
        "Revenue prediction for next 6 months"
    ]
}
overview_df = pd.DataFrame(overview)

# Top 10 high seasonality
top_high = summary[summary['Seasonality Score'] >= 4].sort_values('Seasonality Score', ascending=False).head(10)

# Top 10 stable
top_stable = summary[summary['Seasonality Score'] == 1].head(10)

# Save everything into a single Excel file with multiple sheets
with pd.ExcelWriter('Slide_Deck_Summary_Report.xlsx') as writer:
    overview_df.to_excel(writer, sheet_name='Executive Overview', index=False)
    top_high.to_excel(writer, sheet_name='Top 10 High Seasonality', index=False)
    top_stable.to_excel(writer, sheet_name='Top 10 Stable Venues', index=False)
    summary.to_excel(writer, sheet_name='All Venue Insights', index=False)

print("✅ Slide-style summary saved to 'Slide_Deck_Summary_Report.xlsx'")


✅ Slide-style summary saved to 'Slide_Deck_Summary_Report.xlsx'
